# Introduction

In this tutorial, we will use the **UNI model** from [MahmoodLab](https://github.com/mahmoodlab/UNI) (follow the Installation steps), a self-supervised vision transformer trained on histology slides, to extract **patch-level feature embeddings** from Whole Slide Images (WSIs).

The steps we will follow are:

1. **Load** the pretrained UNI model and its checkpoint weights.
2. **Tokenize** the WSI into small, non-overlapping patches of size **224×224 pixels**.
3. **Save** each patch image and record its spatial location (X, Y coordinates) on the original WSI.
4. **Feed** the patches through the pretrained UNI model to extract deep **feature embeddings**.
5. **Link** each embedding with its spatial metadata (Slide ID, Patch ID, coordinates).
6. **Save** the final dataset for downstream spatial analysis tasks.

---

By following this process, we create a compact and meaningful representation of very large WSIs, which can be easily used for downstream tasks.


In [1]:
#!source ~/.bashrc
#!conda activate UNI

# Imports and Environment Setup

In this section, we import the necessary libraries and set up the environment.

- **PyTorch** (`torch`) and **TorchVision**: for deep learning operations and image handling.
- **Pandas**, **NumPy**: for working with tabular data and arrays.
- **PIL** (`Image`): for opening and processing images.
- **Tifffile**: for reading `.tiff` format images, commonly used in digital pathology.
- **UNI model utilities**: 
  - `get_encoder` to load the pretrained UNI model.
  - `extract_patch_features_from_dataloader` to extract embeddings from patches.
  - Various evaluation utilities for downstream analysis (e.g., linear probe, k-NN, ProtoNet).

Finally, we set the **device** to use a **GPU** if available, otherwise fallback to **CPU**:
```python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
import torch
import torchvision
import os
from os.path import join as j_
from PIL import Image
import pandas as pd
import numpy as np
import tifffile
from uni import get_encoder
import tifffile
import numpy as np
import os
import pandas as pd
from PIL import Image
from uni.downstream.extract_patch_features import extract_patch_features_from_dataloader
from uni.downstream.eval_patch_features.linear_probe import eval_linear_probe
from uni.downstream.eval_patch_features.fewshot import eval_knn, eval_fewshot
from uni.downstream.eval_patch_features.protonet import ProtoNet, prototype_topk_vote
from uni.downstream.eval_patch_features.metrics import get_eval_metrics, print_metrics
from uni.downstream.utils import concat_images
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/rgr/anaconda3/envs/UNI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cpu')

# Input and Output setup

In [5]:
method = "standard_224" #Method to extract the patches
tissue = "adenocarcinoma" #Type of tissue

wsi_folder = f"imgs/{tissue}/" #Where the img is located
output_base_folder = f"{method}/Patches/" #Where you want to save the patches
embedding_file = f"{method}/{tissue}/WSI_patch_embeddings.csv" #name of the file with the embeddings
metadata_file = f"{method}/{tissue}/WSI_patch_metadata.csv"   #name of the fiel(optional)
local_dir = 'checkpoint_path' #Where the model is located

os.makedirs(f"{method}", exist_ok=True)
os.makedirs(f"{method}/{tissue}", exist_ok=True)
os.makedirs(f"{method}/Patches", exist_ok=True)

# Loading the Pretrained UNI Model and Preparing Image Transformations

In this section, we load the **pretrained UNI model** and prepare the **transformations** that will be applied to the image patches before feeding them into the model.

Steps:

1. **Import additional libraries**:
   - `timm`: a popular library that provides implementations of vision transformer models.
   - `huggingface_hub`: for downloading model weights (optional here if already downloaded).

2. **Set the local directory**:
   - `local_dir` is where the pretrained model checkpoint (`pytorch_model.bin`) is stored. Here 'checkpoint_path' is an exampl, set where your checkpoint is located.
   - (Optional) If not downloaded yet, you could use `hf_hub_download` to fetch the weights from Hugging Face.

3. **Model setup**:
   - We define the model architecture using `timm_kwargs` to match the configuration used by UNI (based on a Vision Transformer (ViT) backbone).
   - The model is created with `timm.create_model(**timm_kwargs)`.
   - Pretrained weights are loaded with `model.load_state_dict(...)`.
   - The model is set to evaluation mode (`model.eval()`) and moved to the selected device (CPU or GPU).

4. **Image preprocessing**:
   - Before passing patches to the model, we apply a series of transformations:
     - **Resize** the patch to 224×224 pixels.
     - **Convert** the image to a PyTorch tensor.
     - **Normalize** pixel values using ImageNet statistics (mean and std).



In [6]:
import os
import torch
from torchvision import transforms
import timm
from huggingface_hub import login, hf_hub_download

#login()  

#os.makedirs(local_dir, exist_ok=True)  
#hf_hub_download("MahmoodLab/UNI2-h", filename="pytorch_model.bin", local_dir=local_dir, force_download=True)
timm_kwargs = {'model_name': 'vit_giant_patch14_224',
    'img_size': 224, 
               'patch_size': 14, 
               'depth': 24,
               'num_heads': 24,
               'init_values': 1e-5, 
               'embed_dim': 1536,
               'mlp_ratio': 2.66667*2,
               'num_classes': 0, 
               'no_embed_class': True,
               'mlp_layer': timm.layers.SwiGLUPacked, 
               'act_layer': torch.nn.SiLU, 
               'reg_tokens': 8, 
               'dynamic_img_size': True
              }
model = timm.create_model(**timm_kwargs)
model.load_state_dict(torch.load(os.path.join(local_dir, "pytorch_model.bin"), map_location="cpu"), strict=True)
model.eval()
model.to(device)
transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)


# Tokenizing Whole Slide Images (WSIs) into Patches

In this section, we split each Whole Slide Image (WSI) into small non-overlapping patches and save them as individual image files, along with recording their spatial metadata.

Steps:

1. **Set up paths**:
   - `wsi_folder`: location of original WSI images (in `.tif` format). (set your location)
   - `output_base_folder`: where extracted patches will be saved. (set your location)
   - `metadata_file`: CSV file to store patch metadata (Slide_ID, Patch_ID, X, Y, file path). (set your file name and location or keep it as the default)

2. **Patch parameters**:
   - **Patch size**: `224 × 224` pixels.
   - **Stride**: `224` pixels (non-overlapping patches).

3. **Processing each WSI**:
   - Load the `.tif` file using `tifffile.imread()`.
   - Convert the numpy array to a PIL image for easier patch extraction.
   - Loop over the WSI grid and **crop** patches at regular intervals.
   - Save each patch as a `.png` image.
   - Record metadata for each patch:
     - Slide ID (origin WSI)
     - Patch filename
     - Top-left (X, Y) coordinate of the patch within the slide
     - Path where the patch is stored

4. **Handling errors**:
   - If a WSI fails to load or process, it is added to `failed_wsi_files`.

5. **Metadata saving**:
   - After processing all WSIs, the collected metadata is stored in a single CSV file (`WSI_patch_metadata_prostate.csv`).

---

The metadata will also be important for linking back embeddings to their original spatial locations on the WSI.


In [7]:

patch_size = (224, 224)  
stride = 224  
root_path = output_base_folder


wsi_files = [f for f in os.listdir(wsi_folder) if f.endswith(".tif")]
failed_wsi_files = []
all_patch_metadata = []  

for wsi_file in wsi_files:
    wsi_path = os.path.join(wsi_folder, wsi_file)
    
    try:
    
        wsi_image = tifffile.imread(wsi_path)
        
      
        wsi_image = Image.fromarray(wsi_image)  

  
        slide_name = os.path.splitext(wsi_file)[0]
        slide_output_folder = os.path.join(output_base_folder, slide_name)
        os.makedirs(slide_output_folder, exist_ok=True)

    
        wsi_width, wsi_height = wsi_image.size
        print(f"Processing {slide_name} - Size: {wsi_width}x{wsi_height}")

        saved_patches = 0
        for x in range(0, wsi_width - patch_size[0], stride):
            for y in range(0, wsi_height - patch_size[1], stride):
                patch = wsi_image.crop((x, y, x + patch_size[0], y + patch_size[1]))

               
               
                patch_filename = f"patch_{saved_patches+1}.png"
                patch_path = os.path.join(slide_output_folder, patch_filename)
                patch.save(patch_path)

                
                all_patch_metadata.append([slide_name, patch_filename, x, y, patch_path])
                saved_patches += 1

        print(f"Saved {saved_patches} patches for {slide_name}")

    except Exception as e:
        print(f"Error processing {wsi_file}: {e}")
        failed_wsi_files.append(wsi_file)


metadata_df = pd.DataFrame(all_patch_metadata, columns=['Slide_ID', 'Patch_ID', 'X', 'Y', 'Path'])
metadata_df.to_csv(metadata_file, index=False)

print("All WSIs processed successfully!")
print("Failed WSIs:", failed_wsi_files)
print(f"Metadata saved in {metadata_file}")


Processing Visium_FFPE_Human_Prostate_Cancer_Adenocarcinoma - Size: 27482x25219
Saved 13664 patches for Visium_FFPE_Human_Prostate_Cancer_Adenocarcinoma
All WSIs processed successfully!
Failed WSIs: []
Metadata saved in standard_224/adenocarcinoma/WSI_patch_metadata.csv


# Extracting Patch Feature Embeddings Using the UNI Model

In this section, we load the extracted patches, pass them through the pretrained UNI model, and save their feature embeddings along with spatial metadata.

Steps:

1. **Patch Dataset Preparation**:
   - Define a `PatchDataset` class that:
     - Loads all `.png` patch images from a given folder.
     - Applies the same image transformations we defined earlier.
     - Returns an image and a dummy label (not used here).

2. **Loading Patches and Creating DataLoader**:
   - For each WSI (each folder of patches):
     - Create a `PatchDataset` and corresponding `DataLoader`.
     - Load patches in small batches (batch size = 4) without shuffling (preserve patch order).

3. **Extract Patch Features**:
   - Use the `extract_patch_features_from_dataloader` function from UNI to pass the patches through the model.
   - Extract the feature embeddings for each patch.

4. **Collect Embeddings and Metadata**:
   - For each patch, store:
     - Its extracted embedding.
     - The corresponding Slide_ID (WSI name).
     - The Patch_ID (patch filename).

5. **Merge with Patch Location Metadata**:
   - Combine the extracted embeddings with the previously saved patch spatial information (`X`, `Y` coordinates) using a `match_id` (Slide_ID + Patch_ID).

6. **Save Final Data**:
   - Store everything in a CSV file `WSI_patch_embeddings_prostate.csv`, which contains:
     - Patch embeddings (feature vectors)
     - Slide_ID
     - Patch_ID
     - X, Y coordinates on the original WSI

---

At the end of this step, we will have a **structured table** linking each patch's location to its **deep feature representation**, ready for downstream spatial analysis tasks.


In [8]:
from uni.downstream.extract_patch_features import extract_patch_features_from_dataloader
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
from PIL import Image
import os
import pandas as pd
import torch

class PatchDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = glob.glob(os.path.join(root_dir, "*.png"))  # Load all PNGs
        self.transform = transform
        self.patch_ids = [os.path.basename(img) for img in self.image_paths]  # Store Patch_IDs

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure 3 channels

        if self.transform:
            image = self.transform(image)
            
        return image, -1  




valid_folders = [f for f in sorted(os.listdir(root_path)) if os.path.isdir(os.path.join(root_path, f)) and not f.startswith(".")]

all_embeddings = []
all_sample_ids = []
all_patch_ids = []  

for folder in valid_folders:
    folder_path = os.path.join(root_path, folder)

    test_dataset = PatchDataset(root_dir=folder_path, transform=transform)
    test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

   
    test_features = extract_patch_features_from_dataloader(model, test_dataloader)
    test_feats = torch.Tensor(test_features['embeddings'])

    
    all_embeddings.append(test_feats.numpy())  # Convert to NumPy
    all_sample_ids.extend([folder] * test_feats.shape[0])  # Assign folder name to each patch
    all_patch_ids.extend(test_dataset.patch_ids)  


df_embeddings = pd.DataFrame(
    torch.cat([torch.tensor(arr) for arr in all_embeddings], dim=0).numpy()  # Flatten list of arrays
)
df_embeddings["Slide_ID"] = all_sample_ids 
df_embeddings["Patch_ID"] = all_patch_ids  


df_embeddings['match_id']=df_embeddings['Slide_ID']+'_'+df_embeddings['Patch_ID']
metadata_df['match_id']=metadata_df['Slide_ID']+'_'+metadata_df['Patch_ID']
df_embeddings=df_embeddings.merge(metadata_df[['match_id','X','Y','Patch_ID']],on='match_id')
df_embeddings.to_csv(embedding_file,index=False) #final file

  0%|          | 3/3416 [00:21<6:41:00,  7.05s/it]


KeyboardInterrupt: 